<a href="https://colab.research.google.com/github/AB-912/Text-Classification/blob/main/Text_Classfication_Using_Tensorflow_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALLING TENSORFLOW AND IMPORT LIBRARIES AND CHECKING FOR GPU'S,VERSION OF TENSORFLOW,VERSION OF HUB**


In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("VERSION : ",tf.__version__)
print("EAGER MODE : ",tf.executing_eagerly())
print("HUB VERSION : ", hub.__version__)
print("GPU IS ", "AVAILABLE" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


VERSION :  2.8.2
EAGER MODE :  True
HUB VERSION :  0.12.0
GPU IS  AVAILABLE


### **DOWNLOADING IMDB DATASET using load function and then splitting the dataset in train and validation(60%) and test(40%)**

In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

# **EXPLORING THE DATASET**

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

**PRINTING FIRST TEN LINES AND THEN CHECKING THE LABELS**

1.   **0 - FOR NEGATIVE REVIEW**
2.   **1 - FOR POSITIVE REVIEW**



In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

# **BUILDING A MODEL using a  pre-trained text embedding model from TensorFlow**

In [6]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


# **LOSS FUNCTION AND OPTIMIZER**

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# **TRAINING THE MODEL**

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


30/30 [==============================] - 4s 73ms/step - loss: 0.6346 - accuracy: 0.6538 - val_loss: 0.5759 - val_accuracy: 0.7352
Epoch 2/20
30/30 [==============================] - 2s 54ms/step - loss: 0.4984 - accuracy: 0.8133 - val_loss: 0.4659 - val_accuracy: 0.8138
Epoch 3/20
30/30 [==============================] - 3s 80ms/step - loss: 0.3653 - accuracy: 0.8765 - val_loss: 0.3813 - val_accuracy: 0.8453
Epoch 4/20
30/30 [==============================] - 2s 68ms/step - loss: 0.2626 - accuracy: 0.9145 - val_loss: 0.3360 - val_accuracy: 0.8590
Epoch 5/20
30/30 [==============================] - 2s 56ms/step - loss: 0.1924 - accuracy: 0.9421 - val_loss: 0.3167 - val_accuracy: 0.8660
Epoch 6/20
30/30 [==============================] - 2s 53ms/step - loss: 0.1419 - accuracy: 0.9617 - val_loss: 0.3097 - val_accuracy: 0.8723
Epoch 7/20
30/30 [==============================] - 2s 52ms/step - loss: 0.1042 - accuracy: 0.9762 - val_loss: 0.3135 - val_accuracy: 0.8698
Epoch 8/20
30/30 [======

# **EVALUATING THE MODEL**

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.4938 - accuracy: 0.8475 - 2s/epoch - 35ms/step
loss: 0.494
accuracy: 0.847
